# DWH

## Import

In [ ]:
import os
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from glob import glob
from sqlalchemy import create_engine

Eine importierte Stopword Liste aus Github wird verwendet.

In [ ]:
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:]

## HTML und CSV Parsing Module

Dieses Modul enthält Funktionen zum Einlesen, Verarbeiten und Analysieren von HTML-Dateien, die in CSV-Dateien gelistet sind. Die Hauptfunktionen umfassen das Lesen von HTML-Dateien, die Textverarbeitung, das Filtern von Wörtern nach bestimmten Schlüsselwörtern, das Zählen von Wortvorkommen und das Aggregieren der Ergebnisse aus mehreren CSV-Dateien.

In [1]:
def read_html_file(file_name, encoding="utf-8"):
    # Öffne und lese die HTML-Datei
    with open(file_name, "r", encoding=encoding) as f:
        text = f.read()
    return text

def process_html(text):
    # Extrahiere Text und filtere Wörter
    text = BeautifulSoup(text, "html.parser").get_text()
    items = text.replace("\n", " ").replace("\t", " ").lower().split(" ")
    items = [i for i in items if len(i) > 2 and i not in stopwords_list]
    return items

def parse_html(name, date, file_name, encoding="utf-8"):
    # Lese und verarbeite HTML, zähle Wörter
    content = read_html_file(file_name, encoding)
    soup = BeautifulSoup(content, "html.parser")
    text = soup.get_text()
    items = process_html(text)
    item_count = pd.Series(items).value_counts()
    count = item_count.to_frame(name="count").reset_index().rename(columns={"index": "word"})
    count["date"] = date
    count["paper"] = name
    return count

def parse_csv_files_in_directory(directory):
    # Verarbeite alle CSV-Dateien im Verzeichnis
    result_list = []  
    csv_filename_list = glob(os.path.join(directory, "*.csv"))
    
    for csv_file in csv_filename_list:
        df = pd.read_csv(csv_file).dropna()
        for _, row in df.iterrows():
            name = row["name"]
            file_name = row["file_name"]
            date = row["date"]
            encoding = row.get("encoding", "utf-8")
    
            if os.path.exists(file_name):
                # Verarbeite HTML-Datei und füge Ergebnis hinzu
                count = parse_html(name, date, file_name, encoding)
                result_list.append(count)
            else:
                print(f"Datei nicht gefunden: {file_name}")
    
    if result_list:
        # Kombiniere alle Ergebnisse
        return pd.concat(result_list, ignore_index=True)
    else:
        return pd.DataFrame()

# Funktion um nach bestimmten Wörtern zu filtern
def filter_words(df, column, keywords):
    
    def extract_keyword(text):
        for keyword in keywords:
            if keyword in text:
                return keyword
        return None  

    
    df[column] = df[column].apply(extract_keyword)
    return df


## CSV-Dateien Filtern und Aktualisieren

Dieser Abschnitt des Moduls durchsucht ein angegebenes Verzeichnis nach CSV-Dateien, filtert die Zeilen basierend auf einer vordefinierten Liste von Namen (`behalten`) und aktualisiert die CSV-Dateien entsprechend. Wenn eine CSV-Datei keine `'name'`-Spalte enthält oder nach dem Filtern keine relevanten Einträge mehr vorhanden sind, wird eine entsprechende Meldung ausgegeben.


In [ ]:
# Verzeichnis, in dem sich die CSV-Dateien befinden
DATA_LAKE_DIR = "input/data-lake"

# Liste der Namen, die in den CSV-Dateien beibehalten werden sollen
behalten = [
    "sz", "zeit", "faz", "heise", "golem", "tagesspiegel", "taz",
    "abendblatt", "berliner", "welt", "esslinger", "ntv", "pioneer",
    "suedwest", "uebermedien", "dlf", "spiegel", "stern",
    "tagesschau", "wiwo"
]
# Durchlaufe alle Dateien im angegebenen Daten-Lake-Verzeichnis
for file_name in os.listdir(DATA_LAKE_DIR):
    # Überprüfe, ob die Datei die Endung '.csv' hat
    if file_name.endswith(".csv"):
        # Erstelle den vollständigen Pfad zur aktuellen CSV-Datei
        file_path = os.path.join(DATA_LAKE_DIR, file_name)
        
        # CSV-Datei einlesen
        df = pd.read_csv(file_path)
        
        # Überprüfen, ob die Spalte 'name' in der CSV-Datei vorhanden ist
        if "name" in df.columns:
            # Filtere die DataFrame-Zeilen, sodass nur die enthalten sind, deren 'name' in der Liste 'behalten' ist
            df = df[df["name"].isin(behalten)]
            
            # Speichere die gefilterte DataFrame zurück in die ursprüngliche CSV-Datei ohne den Index
            df.to_csv(file_path, index=False)
        else:
            # Ausgabe einer Meldung, falls die 'name'-Spalte nicht vorhanden ist
            print(f"Die Datei ist leer oder enthält keine 'name'-Spalte: {file_name}")


## HTML-Dateien Lesen und Verarbeiten

Dieser Codeabschnitt verarbeitet die aggregierten Wortzählungen aus den CSV-Dateien, filtert sie nach den definierten Parteien und bereitet das Ergebnis für weitere Analysen vor.

In [ ]:
# Verarbeite CSV-Dateien und erhalte das aggregierte Ergebnis
result = parse_csv_files_in_directory("input/data-lake")

# Liste der zu filternden Parteien
keywords = ['spd', 'cdu', 'afd', 'fdp', 'grünen', 'linke']

# Filtere das DataFrame basierend auf den Schlüsselwörtern in der Spalte 'word'
filtered_df = filter_words(result, 'word', keywords)

# Entferne fehlende Werte und setze den Index zurück
result = result.dropna().reset_index(drop=True)

# Erstelle eine Kopie des bereinigten DataFrames für weitere Analysen
party_df = result.copy()

# Zeige die ersten fünf Zeilen des party_df an
party_df.head()


## Zwischenwahlergebnisse aus dem Polibarometer einlesen

Dieser Codeabschnitt lädt die Wahlergebnisse aus einer CSV-Datei, benennt die Spalten um und formatiert das Datum.

In [ ]:
# Lese die CSV-Datei 'wahlen.csv' mit ';' als Trennzeichen und wähle spezifische Spalten aus
wahlen_df = pd.read_csv('wahlen.csv', delimiter=';', usecols=[0,1,2,3,4,5,6], parse_dates=[1], dayfirst=True)

# Benenne die Spalten für bessere Lesbarkeit um
wahlen_df.columns = ['date', 'CDU', 'SPD', 'Grüne', 'FDP', 'Linke', 'AfD']

# Konvertiere die 'date'-Spalte in das Datetime-Format
wahlen_df['date'] = pd.to_datetime(wahlen_df['date'], format='%d.%m.%y')  

# Zeige die ersten fünf Zeilen des DataFrames an
wahlen_df.head() 

## sql verbindung

In [ ]:
engine = create_engine("sqlite:////../temp/meine_datenbank.db")
party_df.to_sql("party", con=engine, if_exists="replace", index=False)
wahlen_df.to_sql("wahlen", con=engine, if_exists="replace", index=False)